<a href="https://colab.research.google.com/github/Guiga1993/engenharia-de-software_fork/blob/main/Fault_Code_Extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install streamlit plotly pandas
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
changed 22 packages in 2s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏

In [13]:
%%writefile app.py

Writing app.py


In [ ]:


import streamlit as st
import pandas as pd
import plotly.express as px

st.set_page_config(layout="wide", page_title="Fault Analysis Dashboard")

st.title("🛠️ Fault Code Interactive Explorer")

# 1. File Uploader
uploaded_file = st.sidebar.file_uploader("Upload your Fault Log (CSV)", type="csv")

if uploaded_file:
    df = pd.read_csv(uploaded_file)

    # Sidebar Filters
    st.sidebar.header("Filters")

    # Filter by Compartment Frequency
    comp_counts = df['Compartment Name'].value_counts()
    min_faults = st.sidebar.slider("Minimum faults per compartment", 1, int(comp_counts.max()), 1)

    # Identify compartments that meet the threshold
    top_compartments = comp_counts[comp_counts >= min_faults].index.tolist()

    # Filter by Fault ID
    all_faults = df['Fault ID'].unique().tolist()
    selected_faults = st.sidebar.multiselect("Filter by Fault ID", all_faults, default=all_faults)

    # Apply Filters to Data
    filtered_df = df[
        (df['Compartment Name'].isin(top_compartments)) &
        (df['Fault ID'].isin(selected_faults))
    ]

    # 2. Visualizations
    col1, col2 = st.columns(2)

    with col1:
        st.subheader("Fault Distribution by Compartment")
        # Treemap is great for seeing "The most faults" visually
        fig_tree = px.treemap(filtered_df,
                             path=['Compartment Name', 'Component name', 'Fault ID'],
                             color='Fault severity',
                             color_continuous_scale='Reds')
        st.plotly_chart(fig_tree, use_container_width=True)

    with col2:
        st.subheader("Severity vs. Component")
        fig_sun = px.sunburst(filtered_df,
                             path=['Compartment Name', 'Component name'],
                             values='Fault severity',
                             color='Fault severity',
                             color_continuous_scale='Viridis')
        st.plotly_chart(fig_sun, use_container_width=True)

    # 3. Raw Data View
    st.subheader("Filtered Data Reference")
    st.write(filtered_df)

else:
    st.info("Please upload a CSV file to begin.")